# Manipulating Data in DataFrames HW


#### Let's get started applying what we learned in the lecure!

I've provided several questions below to help test and expand you knowledge from the code along lecture. So let's see what you've got!

First create your spark instance as we need to do at the start of every project.

In [35]:
%config Completer.use_jedi = False

import os
import sys
import shutil
import pandas as pd

pd.set_option('display.max_colwidth', None)

BASE_DIR = os.path.realpath(os.path.join(os.getcwd(), "..", ".."))

if not BASE_DIR in sys.path:
    sys.path.append(BASE_DIR)
    
from utils import extract_zip

DATASETS_PATH = "datasets/"

## Read in our Republican vs. Democrats Tweet DataFrame

Attached to the lecture

In [102]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

spark = SparkSession.builder.appName("ManipulateHW").getOrCreate()

# Allow use single slash when using raw string (r"...")
# spark.sql("SET spark.sql.parser.escapedStringLiterals=true")

# Fix the Date conversion problem - REVIEW THE DEFINITIVE SOLUTION LATER!!!
spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY")

spark

## About this dataframe

Extracted tweets from all of the representatives (latest 200 as of May 17th 2018)

**Source:** https://www.kaggle.com/kapastor/democratvsrepublicantweets#ExtractedTweets.csv

Use either .show() or .toPandas() check out the first view rows of the dataframe to get an idea of what we are working with.

In [4]:
datasets = extract_zip(
    zip_file=os.path.join(DATASETS_PATH, "republican_vs_democrats.zip")
)

datasets

'/tmp/aaaaa-rmlrw2m5'

In [40]:
tweets = spark.read.csv(
    os.path.join(datasets, "ExtractedTweets.csv"), 
    inferSchema=True, 
    header=True
)
tweets.show(5, truncate=True)

+--------------------+-------------+--------------------+
|               Party|       Handle|               Tweet|
+--------------------+-------------+--------------------+
|            Democrat|RepDarrenSoto|Today, Senate Dem...|
|            Democrat|RepDarrenSoto|RT @WinterHavenSu...|
|            Democrat|RepDarrenSoto|RT @NBCLatino: .@...|
|Congress has allo...|         null|                null|
|            Democrat|RepDarrenSoto|RT @NALCABPolicy:...|
+--------------------+-------------+--------------------+
only showing top 5 rows



**Prevent Truncation of view**

If the view you produced above truncated some of the longer tweets, see if you can prevent that so you can read the whole tweet.

In [6]:
tweets.show(5, truncate=False)

+----------------------------------+-------------+--------------------------------------------------------------------------------------------------------------------------------------------+
|Party                             |Handle       |Tweet                                                                                                                                       |
+----------------------------------+-------------+--------------------------------------------------------------------------------------------------------------------------------------------+
|Democrat                          |RepDarrenSoto|Today, Senate Dems vote to #SaveTheInternet. Proud to support similar #NetNeutrality legislation here in the House… https://t.co/n3tggDLU1L |
|Democrat                          |RepDarrenSoto|RT @WinterHavenSun: Winter Haven resident / Alta Vista teacher is one of several recognized by @RepDarrenSoto for National Teacher Apprecia…|
|Democrat                          |RepD

In [7]:
tweets.limit(5).toPandas()

,Party,Handle,Tweet
0,Democrat,RepDarrenSoto,"Today, Senate Dems vote to #SaveTheInternet. Proud to support similar #NetNeutrality legislation here in the House… https://t.co/n3tggDLU1L"
1,Democrat,RepDarrenSoto,RT @WinterHavenSun: Winter Haven resident / Alta Vista teacher is one of several recognized by @RepDarrenSoto for National Teacher Apprecia…
2,Democrat,RepDarrenSoto,RT @NBCLatino: .@RepDarrenSoto noted that Hurricane Maria has left approximately $90 billion in damages.
3,"Congress has allocated about $18…""",None,None
4,Democrat,RepDarrenSoto,RT @NALCABPolicy: Meeting with @RepDarrenSoto . Thanks for taking the time to meet with @LatinoLeader ED Marucci Guzman. #NALCABPolicy2018.…


**Print Schema**

First, check the schema to make sure the datatypes are accurate. 

In [8]:
tweets.printSchema()

root
 |-- Party: string (nullable = true)
 |-- Handle: string (nullable = true)
 |-- Tweet: string (nullable = true)



## 1. Can you identify any tweet that mentions the handle @LatinoLeader using regexp_extract?

It doesn't matter how you identify the row, any identifier will do. You can test your script on row 5 from this dataset. That row contains @LatinoLeader. 

In [9]:
(
    tweets
    .select(
        "Tweet", regexp_extract(
            col("Tweet"), "@LatinoLeader", 0
        ).alias("selected")
    )
    .limit(5)
    .toPandas()
)

,Tweet,selected
0,"Today, Senate Dems vote to #SaveTheInternet. Proud to support similar #NetNeutrality legislation here in the House… https://t.co/n3tggDLU1L",
1,RT @WinterHavenSun: Winter Haven resident / Alta Vista teacher is one of several recognized by @RepDarrenSoto for National Teacher Apprecia…,
2,RT @NBCLatino: .@RepDarrenSoto noted that Hurricane Maria has left approximately $90 billion in damages.,
3,None,None
4,RT @NALCABPolicy: Meeting with @RepDarrenSoto . Thanks for taking the time to meet with @LatinoLeader ED Marucci Guzman. #NALCABPolicy2018.…,@LatinoLeader


In [10]:
tweets.withColumn(
    "Latino_Mentions", 
    regexp_extract(tweets.Tweet, "(.)(@LatinoLeader)(.)", 2)
).show(5)

+--------------------+-------------+--------------------+---------------+
|               Party|       Handle|               Tweet|Latino_Mentions|
+--------------------+-------------+--------------------+---------------+
|            Democrat|RepDarrenSoto|Today, Senate Dem...|               |
|            Democrat|RepDarrenSoto|RT @WinterHavenSu...|               |
|            Democrat|RepDarrenSoto|RT @NBCLatino: .@...|               |
|Congress has allo...|         null|                null|           null|
|            Democrat|RepDarrenSoto|RT @NALCABPolicy:...|  @LatinoLeader|
+--------------------+-------------+--------------------+---------------+
only showing top 5 rows



## 2. Replace any value other than 'Democrate' or 'Republican' with 'Other' in the Party column.

We can see from the output below, that there are several other values other than 'Democrate' or 'Republican' in the Part column. We are assuming that this is dirty data that needs to be cleaned up.

In [11]:
counts = tweets.groupBy("Party").count()
counts.orderBy(desc("count")).show(6)

+--------------------+-----+
|               Party|count|
+--------------------+-----+
|          Republican|44392|
|            Democrat|42068|
|            That’s…"|   28|
|https://t.co/oc6J...|   22|
|                 Now|   17|
|               Today|   13|
+--------------------+-----+
only showing top 6 rows



In [12]:
clean = (
    tweets.withColumn(
        "Party",
        when(tweets.Party == "Democrat", "Democrat")
        .when(tweets.Party == "Republican", "Republican")
        .otherwise("Other")
    )
)
counts = clean.groupBy("Party").count()
counts.orderBy(desc("count")).show(6)

+----------+-----+
|     Party|count|
+----------+-----+
|Republican|44392|
|  Democrat|42068|
|     Other| 6029|
+----------+-----+



In [18]:
clean = (
    tweets.select(
        when(tweets.Party == "Democrat", tweets.Party)
        .when(tweets.Party == "Republican", tweets.Party)
        .otherwise("other")
        .alias("Party")
    )
)
counts = clean.groupBy("Party").count()
counts.orderBy(desc("count")).show(6)

+----------+-----+
|     Party|count|
+----------+-----+
|Republican|44392|
|  Democrat|42068|
|     other| 6029|
+----------+-----+



In [13]:
clean = (
    tweets.selectExpr(
        "CASE Party"
        "  WHEN 'Democrat'   THEN Party"
        "  WHEN 'Republican' THEN Party"
        "  ELSE                   'Other' "
        "END AS Party"
    )
)
counts = clean.groupBy("Party").count()
counts.orderBy(desc("count")).show(6)

+----------+-----+
|     Party|count|
+----------+-----+
|Republican|44392|
|  Democrat|42068|
|     Other| 6029|
+----------+-----+



In [19]:
clean = (
    tweets.selectExpr(
        "CASE "
        "  WHEN Party IN ('Democrat', 'Republican') THEN Party "
        "  ELSE                                          'Other' "
        "END AS Party"
    )
)
counts = clean.groupBy("Party").count()
counts.orderBy(desc("count")).show(6)

+----------+-----+
|     Party|count|
+----------+-----+
|Republican|44392|
|  Democrat|42068|
|     Other| 6029|
+----------+-----+



## 3. Delete all embedded links (ie. "https:....)

For example see the first row in the tweets dataframe. 

*Note: this may require an google search :)*

In [26]:
pattern = (
    "(https?|ftp)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?"
)

(
    tweets.select(
        "Tweet", regexp_replace(col("Tweet"), pattern, " ").alias("Cleaned")
    )
    .limit(5)
    .toPandas()
)

,Tweet,Cleaned
0,"Today, Senate Dems vote to #SaveTheInternet. Proud to support similar #NetNeutrality legislation here in the House… https://t.co/n3tggDLU1L","Today, Senate Dems vote to #SaveTheInternet. Proud to support similar #NetNeutrality legislation here in the House…"
1,RT @WinterHavenSun: Winter Haven resident / Alta Vista teacher is one of several recognized by @RepDarrenSoto for National Teacher Apprecia…,RT @WinterHavenSun: Winter Haven resident / Alta Vista teacher is one of several recognized by @RepDarrenSoto for National Teacher Apprecia…
2,RT @NBCLatino: .@RepDarrenSoto noted that Hurricane Maria has left approximately $90 billion in damages.,RT @NBCLatino: .@RepDarrenSoto noted that Hurricane Maria has left approximately $90 billion in damages.
3,None,None
4,RT @NALCABPolicy: Meeting with @RepDarrenSoto . Thanks for taking the time to meet with @LatinoLeader ED Marucci Guzman. #NALCABPolicy2018.…,RT @NALCABPolicy: Meeting with @RepDarrenSoto . Thanks for taking the time to meet with @LatinoLeader ED Marucci Guzman. #NALCABPolicy2018.…


## 4. Remove any leading or trailing white space in the tweet column

In [27]:
tweets.select("Tweet", trim(col("Tweet")).alias("Trimed")).show(1, truncate=False)

+-------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------+
|Tweet                                                                                                                                      |Trimed                                                                                                                                     |
+-------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------+
|Today, Senate Dems vote to #SaveTheInternet. Proud to support similar #NetNeutrality legislation here in the House… https://t.co/n3tggDLU1L|Today, Senate

## 5. Rename the 'Party' column to 'Dem_Rep'

No real reason here :) just wanted you to get practice doing this. 

In [30]:
renamed = tweets.withColumnRenamed("Party", "Dem_Rep")
renamed.limit(5).toPandas()

,Dem_Rep,Handle,Tweet
0,Democrat,RepDarrenSoto,"Today, Senate Dems vote to #SaveTheInternet. Proud to support similar #NetNeutrality legislation here in the House… https://t.co/n3tggDLU1L"
1,Democrat,RepDarrenSoto,RT @WinterHavenSun: Winter Haven resident / Alta Vista teacher is one of several recognized by @RepDarrenSoto for National Teacher Apprecia…
2,Democrat,RepDarrenSoto,RT @NBCLatino: .@RepDarrenSoto noted that Hurricane Maria has left approximately $90 billion in damages.
3,"Congress has allocated about $18…""",None,None
4,Democrat,RepDarrenSoto,RT @NALCABPolicy: Meeting with @RepDarrenSoto . Thanks for taking the time to meet with @LatinoLeader ED Marucci Guzman. #NALCABPolicy2018.…


In [29]:
tweets.select(col("Party").alias("Dem_Rep")).show(5)

+--------------------+
|             Dem_Rep|
+--------------------+
|            Democrat|
|            Democrat|
|            Democrat|
|Congress has allo...|
|            Democrat|
+--------------------+
only showing top 5 rows



## 6. Concatenate the Party and Handle columns

Silly yes... but good practice.

pyspark.sql.functions.concat_ws(sep, *cols)[source] <br>
Concatenates multiple input string columns together into a single string column, using the given separator.

In [31]:
tweets.select(concat_ws(" @ ", "Party", "Handle")).show(5, truncate=False)

+----------------------------------+
|concat_ws( @ , Party, Handle)     |
+----------------------------------+
|Democrat @ RepDarrenSoto          |
|Democrat @ RepDarrenSoto          |
|Democrat @ RepDarrenSoto          |
|Congress has allocated about $18…"|
|Democrat @ RepDarrenSoto          |
+----------------------------------+
only showing top 5 rows



## Challenge Question

Let's image that we want to analyze the hashtags that are used in these tweets. Can you extract all the hashtags you see?

In [58]:
tweets.select("Tweet").where("Tweet LIKE '%#%'").show(5, truncate=False)

+--------------------------------------------------------------------------------------------------------------------------------------------+
|Tweet                                                                                                                                       |
+--------------------------------------------------------------------------------------------------------------------------------------------+
|Today, Senate Dems vote to #SaveTheInternet. Proud to support similar #NetNeutrality legislation here in the House… https://t.co/n3tggDLU1L |
|RT @NALCABPolicy: Meeting with @RepDarrenSoto . Thanks for taking the time to meet with @LatinoLeader ED Marucci Guzman. #NALCABPolicy2018.…|
|RT @Tharryry: I am delighted that @RepDarrenSoto will be voting for the CRA to overrule the FCC and save our #NetNeutrality rules. Find out…|
|RT @RepStephMurphy: Great joining @WeAreUnidosUS and @RepDarrenSoto for a roundtable in #Orlando on federal issues affecting central Florid…|

In [71]:
selected = tweets.select("Tweet").where("Tweet LIKE '%#%'")

pattern = r"(#[\w-]+)"

selected.show(5, truncate=False)

hashes = tweets.select(
    regexp_replace("Tweet", fr"[^#]*{pattern}", "$1,").alias("Tweet")
).where(
    "Tweet LIKE '%#%'"
)
hashes.show(5, truncate=False)

hashes = hashes.select(
    regexp_replace("Tweet", fr"{pattern}[^#]*", "$1,").alias("Tweet")
)
hashes.show(5, truncate=False)

hashes = hashes.withColumn("Tweet", split("Tweet", r","))
hashes.show(5, truncate=False)

+--------------------------------------------------------------------------------------------------------------------------------------------+
|Tweet                                                                                                                                       |
+--------------------------------------------------------------------------------------------------------------------------------------------+
|Today, Senate Dems vote to #SaveTheInternet. Proud to support similar #NetNeutrality legislation here in the House… https://t.co/n3tggDLU1L |
|RT @NALCABPolicy: Meeting with @RepDarrenSoto . Thanks for taking the time to meet with @LatinoLeader ED Marucci Guzman. #NALCABPolicy2018.…|
|RT @Tharryry: I am delighted that @RepDarrenSoto will be voting for the CRA to overrule the FCC and save our #NetNeutrality rules. Find out…|
|RT @RepStephMurphy: Great joining @WeAreUnidosUS and @RepDarrenSoto for a roundtable in #Orlando on federal issues affecting central Florid…|

# Let's create our own dataset to work with real dates

This is a dataset of patient visits from a medical office. It contains the patients first and last names, date of birth, and the dates of their first 3 visits. 

In [94]:
from pyspark.sql.types import *

md_office = [
    ('Mohammed','Alfasy','1987-4-8','2016-1-7','2017-2-3','2018-3-2'),
    ('Marcy','Wellmaker','1986-4-8','2015-1-7','2017-1-3','2018-1-2'),
    ('Ginny','Ginger','1986-7-10','2014-8-7','2015-2-3','2016-3-2'),
    ('Vijay','Doberson','1988-5-2','2016-1-7','2018-2-3','2018-3-2'),
    ('Orhan','Gelicek','1987-5-11','2016-5-7','2017-1-3','2018-9-2'),
    ('Sarah','Jones','1956-7-6','2016-4-7','2017-8-3','2018-10-2'),
    ('John','Johnson','2017-10-12','2018-1-2','2018-10-3','2018-3-2')
]

# schema=final_struc
df = spark.createDataFrame(
    md_office,['first_name','last_name','dob','visit1','visit2','visit3']
)

# Check to make sure it worked
df.show()

df.printSchema()

+----------+---------+----------+--------+---------+---------+
|first_name|last_name|       dob|  visit1|   visit2|   visit3|
+----------+---------+----------+--------+---------+---------+
|  Mohammed|   Alfasy|  1987-4-8|2016-1-7| 2017-2-3| 2018-3-2|
|     Marcy|Wellmaker|  1986-4-8|2015-1-7| 2017-1-3| 2018-1-2|
|     Ginny|   Ginger| 1986-7-10|2014-8-7| 2015-2-3| 2016-3-2|
|     Vijay| Doberson|  1988-5-2|2016-1-7| 2018-2-3| 2018-3-2|
|     Orhan|  Gelicek| 1987-5-11|2016-5-7| 2017-1-3| 2018-9-2|
|     Sarah|    Jones|  1956-7-6|2016-4-7| 2017-8-3|2018-10-2|
|      John|  Johnson|2017-10-12|2018-1-2|2018-10-3| 2018-3-2|
+----------+---------+----------+--------+---------+---------+

root
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- dob: string (nullable = true)
 |-- visit1: string (nullable = true)
 |-- visit2: string (nullable = true)
 |-- visit3: string (nullable = true)



Oh no! The dates are still stored as text... let's try converting them again and see if we have any issues this time.

In [104]:
df = (
    df
    .withColumn("dob", to_date(col("dob"), "yyyy-MM-dd"))
    .withColumn("visit1", to_date(col("visit1"), "yyyy-MM-dd"))
    .withColumn("visit2", to_date(col("visit2"), "yyyy-MM-dd"))
    .withColumn("visit3", to_date(col("visit3"), "yyyy-MM-dd"))
)

df.printSchema()

df.show(5, truncate=False)

root
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- dob: date (nullable = true)
 |-- visit1: date (nullable = true)
 |-- visit2: date (nullable = true)
 |-- visit3: date (nullable = true)

+----------+---------+----------+----------+----------+----------+
|first_name|last_name|dob       |visit1    |visit2    |visit3    |
+----------+---------+----------+----------+----------+----------+
|Mohammed  |Alfasy   |1987-04-08|2016-01-07|2017-02-03|2018-03-02|
|Marcy     |Wellmaker|1986-04-08|2015-01-07|2017-01-03|2018-01-02|
|Ginny     |Ginger   |1986-07-10|2014-08-07|2015-02-03|2016-03-02|
|Vijay     |Doberson |1988-05-02|2016-01-07|2018-02-03|2018-03-02|
|Orhan     |Gelicek  |1987-05-11|2016-05-07|2017-01-03|2018-09-02|
+----------+---------+----------+----------+----------+----------+
only showing top 5 rows



## 7. Can you calculate a variable showing the length of time between patient visits?

Compare visit1 to visit2 and visit2 to visit3 for all patients and see what the average length of time is between visits. Create an alias for it as well. 

In [135]:
df.select(
    "dob",
    "visit1",
    "visit2",
    "visit3",
    datediff("visit2", "visit1").alias("delta1"),
    datediff("visit3", "visit2").alias("delta2")
).show(5, truncate=False)

+----------+----------+----------+----------+-----+-----+
|dob       |visit1    |visit2    |visit3    |v2-v1|v3-v2|
+----------+----------+----------+----------+-----+-----+
|1987-04-08|2016-01-07|2017-02-03|2018-03-02|393  |392  |
|1986-04-08|2015-01-07|2017-01-03|2018-01-02|727  |364  |
|1986-07-10|2014-08-07|2015-02-03|2016-03-02|180  |393  |
|1988-05-02|2016-01-07|2018-02-03|2018-03-02|758  |27   |
|1987-05-11|2016-05-07|2017-01-03|2018-09-02|241  |607  |
+----------+----------+----------+----------+-----+-----+
only showing top 5 rows



## 8. Can you calculate the age of each patient?

In [144]:
from datetime import datetime

today = datetime.today().date()

df.select(
    "dob", format_number(
        datediff(to_date(lit(today), "yyyy-mm-dd"), df["dob"]) / 365, 1
    ).alias("age")
).show(5, truncate=False)

+----------+----+
|dob       |age |
+----------+----+
|1987-04-08|33.9|
|1986-04-08|34.9|
|1986-07-10|34.7|
|1988-05-02|32.9|
|1987-05-11|33.9|
+----------+----+
only showing top 5 rows



## 9. Can you extract the month from the first visit column and call it "Month"?

In [145]:
df.select("visit1", month(col("visit1")).alias("Month")).show(5, truncate=False)

+----------+-----+
|visit1    |Month|
+----------+-----+
|2016-01-07|1    |
|2015-01-07|1    |
|2014-08-07|8    |
|2016-01-07|1    |
|2016-05-07|5    |
+----------+-----+
only showing top 5 rows



## 10. Challenges with working with date and timestamps

Let's read in the supermarket sales dataframe attached to the lecture now and see some of the issues that can come up when working with date and timestamps values.

In [148]:
datasets = extract_zip(
    zip_file=os.path.join(DATASETS_PATH, "supermarket_sales.csv.zip")
)

datasets

'/tmp/aaaaa-rmlrw2m5'

## About this dataset

The growth of supermarkets in most populated cities are increasing and market competitions are also high. The dataset is one of the historical sales of supermarket company which has recorded in 3 different branches for 3 months data. 

 - Attribute information
 - Invoice id: Computer generated sales slip invoice identification number
 - Branch: Branch of supercenter (3 branches are available identified by A, B and C).
 - City: Location of supercenters
 - Customer type: Type of customers, recorded by Members for customers using member card and Normal for without member card.
 - Gender: Gender type of customer
 - Product line: General item categorization groups - Electronic accessories, Fashion accessories, Food and beverages, Health and beauty, Home and lifestyle, Sports and travel
 - Unit price: Price of each product in USD
 - Quantity: Number of products purchased by customer
 - Tax: 5% tax fee for customer buying
 - Total: Total price including tax
 - Date: Date of purchase (Record available from January 2019 to March 2019)
 - Time: Purchase time (10am to 9pm)
 - Payment: Payment used by customer for purchase (3 methods are available – Cash, Credit card and Ewallet)
 - COGS: Cost of goods sold
 - Gross margin percentage: Gross margin percentage
 - Gross income: Gross income
 - Rating: Customer stratification rating on their overall shopping experience (On a scale of 1 to 10)

**Source:** https://www.kaggle.com/aungpyaeap/supermarket-sales

### View dataframe and schema as usual

In [152]:
sales = spark.read.csv(
    os.path.join(datasets, "supermarket_sales.csv"),
    inferSchema=True,
    header=True
)

In [153]:
sales.printSchema()

root
 |-- Invoice ID: string (nullable = true)
 |-- Branch: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Customer type: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Product line: string (nullable = true)
 |-- Unit price: double (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- Tax 5%: double (nullable = true)
 |-- Total: double (nullable = true)
 |-- Date: string (nullable = true)
 |-- Time: string (nullable = true)
 |-- Payment: string (nullable = true)
 |-- cogs: double (nullable = true)
 |-- gross margin percentage: double (nullable = true)
 |-- gross income: double (nullable = true)
 |-- Rating: double (nullable = true)



In [154]:
sales.limit(5).toPandas()

,Invoice ID,Branch,City,Customer type,Gender,Product line,Unit price,Quantity,Tax 5%,Total,Date,Time,Payment,cogs,gross margin percentage,gross income,Rating
0,750-67-8428,A,Yangon,Member,Female,Health and beauty,74.69,7,26.1415,548.9715,1/5/2019,13:08,Ewallet,522.83,4.761905,26.1415,9.1
1,226-31-3081,C,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,3.8200,80.2200,3/8/2019,10:29,Cash,76.40,4.761905,3.8200,9.6
2,631-41-3108,A,Yangon,Normal,Male,Home and lifestyle,46.33,7,16.2155,340.5255,3/3/2019,13:23,Credit card,324.31,4.761905,16.2155,7.4
3,123-19-1176,A,Yangon,Member,Male,Health and beauty,58.22,8,23.2880,489.0480,1/27/2019,20:33,Ewallet,465.76,4.761905,23.2880,8.4
4,373-73-7910,A,Yangon,Normal,Male,Sports and travel,86.31,7,30.2085,634.3785,2/8/2019,10:37,Ewallet,604.17,4.761905,30.2085,5.3


### Convert date field to date type

Looks like we need to convert the date field into a date type. Let's go ahead and do that..

In [159]:
sales = sales.withColumn("Date", to_date(sales["Date"], "MM/d/yyyy"))
sales.limit(5).toPandas()

,Invoice ID,Branch,City,Customer type,Gender,Product line,Unit price,Quantity,Tax 5%,Total,Date,Time,Payment,cogs,gross margin percentage,gross income,Rating
0,750-67-8428,A,Yangon,Member,Female,Health and beauty,74.69,7,26.1415,548.9715,2019-01-05,13:08,Ewallet,522.83,4.761905,26.1415,9.1
1,226-31-3081,C,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,3.8200,80.2200,2019-03-08,10:29,Cash,76.40,4.761905,3.8200,9.6
2,631-41-3108,A,Yangon,Normal,Male,Home and lifestyle,46.33,7,16.2155,340.5255,2019-03-03,13:23,Credit card,324.31,4.761905,16.2155,7.4
3,123-19-1176,A,Yangon,Member,Male,Health and beauty,58.22,8,23.2880,489.0480,2019-01-27,20:33,Ewallet,465.76,4.761905,23.2880,8.4
4,373-73-7910,A,Yangon,Normal,Male,Sports and travel,86.31,7,30.2085,634.3785,2019-02-08,10:37,Ewallet,604.17,4.761905,30.2085,5.3


### How can we extract the month value from the date field?

If you had trouble converting the date field in the previous question think about a more creative solution to extract the month from that field.

In [160]:
sales.select("Date", month("Date").alias("Month")).show(5)

+----------+-----+
|      Date|Month|
+----------+-----+
|2019-01-05|    1|
|2019-03-08|    3|
|2019-03-03|    3|
|2019-01-27|    1|
|2019-02-08|    2|
+----------+-----+
only showing top 5 rows



## 11.0 Working with Arrays

Here is a dataframe of reviews from the movie the Dark Night.

In [200]:
from pyspark.sql.functions import *

values = [(5,'Epic. This is the best movie I have EVER seen'), \
          (4,'Pretty good, but I would have liked to seen better special effects'), \
          (3,'So so. Casting could have been improved'), \
          (5,'The most EPIC movie of the year! Casting was awesome. Special effects were so intense.'), \
          (4,'Solid but I would have liked to see more of the love story'), \
          (5,'THE BOMB!!!!!!!')]
reviews = spark.createDataFrame(values,['rating', 'review_txt'])

reviews.show(6,False)

+------+--------------------------------------------------------------------------------------+
|rating|review_txt                                                                            |
+------+--------------------------------------------------------------------------------------+
|5     |Epic. This is the best movie I have EVER seen                                         |
|4     |Pretty good, but I would have liked to seen better special effects                    |
|3     |So so. Casting could have been improved                                               |
|5     |The most EPIC movie of the year! Casting was awesome. Special effects were so intense.|
|4     |Solid but I would have liked to see more of the love story                            |
|5     |THE BOMB!!!!!!!                                                                       |
+------+--------------------------------------------------------------------------------------+



## 11.1 Let's see if we can create an array off of the review text column and then derive some meaningful results from it.

**But first** we need to clean the rview_txt column to make sure we can get what we need from our analysis later on. So let's do the following:

1. Remove all punctuation
2. lower case everything
3. Remove white space (trim)
3. Then finally, split the string

In [201]:
import re
import string

escaped = re.escape(string.punctuation)

reviews.withColumn(
    "review_txt", regexp_replace(col("review_txt"), fr"[{escaped}]", "")
).show(5, truncate=False)

+------+-----------------------------------------------------------------------------------+
|rating|review_txt                                                                         |
+------+-----------------------------------------------------------------------------------+
|5     |Epic This is the best movie I have EVER seen                                       |
|4     |Pretty good but I would have liked to seen better special effects                  |
|3     |So so Casting could have been improved                                             |
|5     |The most EPIC movie of the year Casting was awesome Special effects were so intense|
|4     |Solid but I would have liked to see more of the love story                         |
+------+-----------------------------------------------------------------------------------+
only showing top 5 rows



In [202]:
# Remove all punctuation
reviews = reviews.withColumn(
    "review_txt", regexp_replace(col("review_txt"), "\p{Punct}", "")
)
reviews.show(5, truncate=False)

+------+-----------------------------------------------------------------------------------+
|rating|review_txt                                                                         |
+------+-----------------------------------------------------------------------------------+
|5     |Epic This is the best movie I have EVER seen                                       |
|4     |Pretty good but I would have liked to seen better special effects                  |
|3     |So so Casting could have been improved                                             |
|5     |The most EPIC movie of the year Casting was awesome Special effects were so intense|
|4     |Solid but I would have liked to see more of the love story                         |
+------+-----------------------------------------------------------------------------------+
only showing top 5 rows



In [203]:
# lower case everything
reviews = reviews.withColumn(
    "review_txt", lower(col("review_txt"))
)
reviews.show(5, truncate=False)

+------+-----------------------------------------------------------------------------------+
|rating|review_txt                                                                         |
+------+-----------------------------------------------------------------------------------+
|5     |epic this is the best movie i have ever seen                                       |
|4     |pretty good but i would have liked to seen better special effects                  |
|3     |so so casting could have been improved                                             |
|5     |the most epic movie of the year casting was awesome special effects were so intense|
|4     |solid but i would have liked to see more of the love story                         |
+------+-----------------------------------------------------------------------------------+
only showing top 5 rows



In [204]:
# Remove white space (trim)
reviews = reviews.withColumn(
    "review_txt", trim(col("review_txt"))
)
reviews.show(5, truncate=False)

+------+-----------------------------------------------------------------------------------+
|rating|review_txt                                                                         |
+------+-----------------------------------------------------------------------------------+
|5     |epic this is the best movie i have ever seen                                       |
|4     |pretty good but i would have liked to seen better special effects                  |
|3     |so so casting could have been improved                                             |
|5     |the most epic movie of the year casting was awesome special effects were so intense|
|4     |solid but i would have liked to see more of the love story                         |
+------+-----------------------------------------------------------------------------------+
only showing top 5 rows



In [205]:
# Then finally, split the string
reviews = reviews.withColumn(
    "review_txt_array", split(col("review_txt"), " ")
)
reviews.show(5, truncate=False)

+------+-----------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------+
|rating|review_txt                                                                         |review_txt_array                                                                                   |
+------+-----------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------+
|5     |epic this is the best movie i have ever seen                                       |[epic, this, is, the, best, movie, i, have, ever, seen]                                            |
|4     |pretty good but i would have liked to seen better special effects                  |[pretty, good, but, i, would, have, liked, to, seen, better, special, effects]                     |
|3     |so so casting could have be

## 11.2 Alright now let's see if we can find which reviews contain the word 'Epic'

In [206]:
epic = reviews.withColumn("review_txt_array", array_contains(col("review_txt_array"), "epic"))
epic.show(5, truncate=False)

+------+-----------------------------------------------------------------------------------+----------------+
|rating|review_txt                                                                         |review_txt_array|
+------+-----------------------------------------------------------------------------------+----------------+
|5     |epic this is the best movie i have ever seen                                       |true            |
|4     |pretty good but i would have liked to seen better special effects                  |false           |
|3     |so so casting could have been improved                                             |false           |
|5     |the most epic movie of the year casting was awesome special effects were so intense|true            |
|4     |solid but i would have liked to see more of the love story                         |false           |
+------+-----------------------------------------------------------------------------------+----------------+
only showi

### That's it! Great Job!